<a href="https://colab.research.google.com/github/superaja/covid19/blob/master/covid19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import requests
import io
import plotly.graph_objects as go
import time, datetime

In [3]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# download all data

# NYC
def downloadPDF(state):
  headers = {
    'User-Agent': "PostmanRuntime/7.15.0",
    'Accept': "*/*",
    'Cache-Control': "no-cache",
    'Postman-Token': "ede07156-3b6d-456e-93a0-c901c61783d5,1bcdac9a-9f34-47b5-a02c-e119854db725",
    'Host': "www1.nyc.gov",
    'accept-encoding': "gzip, deflate",
    'Connection': "keep-alive",
    'cache-control': "no-cache"
    }
  y = datetime.datetime.now()
  if state == "CT":
    f_string = str(y.month)+str(y.day-1)+str(y.year)
    f_string
    file_name = "CTDPHCOVID19summary"+f_string+'.pdf'
    file_url = "https://portal.ct.gov/-/media/Coronavirus/"+file_name
    r = requests.get(file_url, stream = True, headers=headers) 
    with open('/content/drive/My Drive/Colab Notebooks/data/'+file_name,"wb") as pdf: 
      for chunk in r.iter_content(chunk_size=1024): 
        # writing one chunk at a time to pdf file 
        if chunk: 
          pdf.write(chunk) 
  else:
    download_files = {'covid-19-daily-data-summary.pdf': 'nyc-cases.pdf', 'covid-19-daily-data-summary-deaths.pdf':'nyc-deaths.pdf', \
                      'covid-19-daily-data-summary-hospitalizations.pdf': 'nyc-hosp.pdf', 'covid-19-syndromic-surveillance.pdf': 'nyc-surv.pdf'} 
    f_string = str(y.month)+str(y.day-1)+str(y.year) + '-' + str(y.hour)
    url = 'https://www1.nyc.gov/assets/doh/downloads/pdf/imm/'
    for fn in download_files.keys():
      file_url = url+fn
      r = requests.get(file_url, stream = True, headers=headers)
      name = '/content/drive/My Drive/Colab Notebooks/data/' + f_string + '-' + download_files[fn]
      #with open('/content/drive/My Drive/Colab Notebooks/data/'+ f_string + '-'+ fn,"wb") as pdf:
      with open(name,"wb") as pdf: 
        for chunk in r.iter_content(chunk_size=1024): 
          # writing one chunk at a time to pdf file 
          if chunk: 
            pdf.write(chunk)
# JHU
df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/04-06-2020.csv')
df_time_series = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
df_us_time_series = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv')
#df_time_series = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
#df_time_series = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')


In [0]:
downloadPDF("CT")

In [0]:
# Data Prep
df_by_country = df.groupby('Country_Region').sum() 
df_by_country['Country_Region'] = df_by_country.index

In [0]:
# add additional metrics
df_by_country['Not Yet Recovered'] = df_by_country['Confirmed'] - df_by_country['Deaths'] - df_by_country['Recovered']
df_by_country['% Recovered'] = round(df_by_country['Recovered'] / df_by_country['Confirmed'], 2)*100
df_by_country['% Mortality'] = round(df_by_country['Deaths'] / df_by_country['Confirmed'], 2)*100
df_by_country['% Not Yet Recovered'] = round(df_by_country['Not Yet Recovered'] / df_by_country['Confirmed'], 2)*100

In [0]:
# core KPIs

# Core Metrics by Country
def covid_core_metrics(country):
  #country = ['Italy', 'US', 'Spain', 'China', 'Korea, South', 'United Kingdom', 'India']
  #country = list(df_by_country['Country/Region'].unique())
  x = df_by_country[df_by_country['Country_Region'].isin(country)]['Country_Region']
  y = df_by_country[df_by_country['Country_Region'].isin(country)]['% Recovered']
  y1 = df_by_country[df_by_country['Country_Region'].isin(country)]['% Mortality']
  y2 = df_by_country[df_by_country['Country_Region'].isin(country)]['% Not Yet Recovered']
  fig = go.Figure(data=go.Bar(x=x, y=y, name='% Recovered', text=y, textposition='auto', marker_color='green'))
  fig.add_trace(go.Bar(x=x, y=y1, name='% Mortality', text=y1, textposition='auto'))
  fig.add_trace(go.Bar(x=x, y=y2, name='% Not Yet Recovered', text=y2, textposition='auto', marker_color='rgb(144, 202, 249)'))
  fig.show()


In [0]:
# Confirmed Country Case Tracker
df_clean = df_time_series.drop(columns=['Province/State', 'Lat', 'Long'])
df_clean_temp = df_clean.drop(columns=['Country/Region'])
df_by_country['Country/Region'] = df_by_country.index
df_country_largest = df_by_country.nlargest(10, 'Confirmed')
country = list(df_country_largest['Country/Region'].unique())
def days_since_100_confirmed():
  fig = go.Figure()
  fig1 = go.Figure()
  fig_day_cases = go.Figure()
  fig_top_10_confirmed = go.Figure()
  for i in country:
    df_country = df_clean[df_clean['Country/Region'] == i]
    df_country = df_country.groupby('Country/Region').sum()
    country_confirmed = df_country.iloc[0].tolist()  
    country_confirmed_gt_100 = [i for i in country_confirmed[1:] if i>100]
    daily_confirmed = [country_confirmed_gt_100[j+1] - country_confirmed_gt_100[j] for j in range(len(country_confirmed_gt_100)-1)]
    label = [round(i/1000, 2) for i in country_confirmed_gt_100]
    days_since_100 = [i for i in range(1, len(country_confirmed_gt_100))]
    fig.add_trace(go.Scatter(x=days_since_100, y=country_confirmed_gt_100, mode='lines+markers', name=i+ ' Confirmed', \
                             textposition='top left'))
    fig1.add_trace(go.Scatter(x=days_since_100, y=country_confirmed_gt_100, mode='lines+markers', name=i+ ' Confirmed', \
                             textposition='top left'))
    fig_day_cases.add_trace(go.Bar(x=days_since_100, y=daily_confirmed, text=i, name=i + ' Confirmed'))
  
  fig.update_layout(title='Confirmed Cases by Country',
                    xaxis_title='Number of Days since 100 Confirmed Cases',
                    yaxis_title= 'Confirmed Cases',
                    )
  fig1.update_layout(title='Confirmed Cases by Country',
                    xaxis_title='Number of Days since 100 Confirmed Cases',
                    yaxis_title= 'Confirmed Cases - Log ',
                    yaxis_type='log'
                    )
  
  fig_day_cases.update_layout(title='Daily Confirmed Cases by Country',
                    xaxis_title='Number of Days since 100 Confirmed Cases',
                    yaxis_title= 'Daily Confirmed Cases',
                                   )
  fig_top_10_confirmed.add_trace(go.Table(
    columnwidth = [40, 100],
    header=dict(values=['Country', 'Confirmed Cases', 'Deaths','Mortality Rate'],
                fill_color='grey',
                line_color='darkslategray',
                align='center',
                font=dict(color='white')),
    cells=dict(values=[df_country_largest['Country/Region'], df_country_largest['Confirmed'], df_country_largest['Deaths'],\
                       round((df_country_largest['Deaths']/df_country_largest['Confirmed'])*100, 2)],
               fill_color='white',
               line_color='darkslategray',
               align='center'))
)
  fig_top_10_confirmed.update_layout(width=800, height=800)
  
  fig_top_10_confirmed.show()

  fig.show()
  fig1.show()
  fig_day_cases.show()

In [12]:
# Add States

# States
def states_rates():
  latest_data = df_us_time_series[df_us_time_series['date'] == df_us_time_series['date'].max()]
  total_us_confirmed = latest_data.groupby('state')['cases'].sum().sum()
  total_us_deaths= latest_data.groupby('state')['deaths'].sum().sum()
  latest_data['pct_confirmed'] = round((latest_data['cases'] / total_us_confirmed)*100, 2)
  latest_data['pct_deaths'] = round((latest_data['deaths'] / total_us_deaths)*100, 2)
  latest_data['pct_deaths_wrt_confirmed'] = round((latest_data['deaths'] /latest_data['cases'] )*100, 2)
  latest_data.sort_values('pct_deaths', ascending=False)
  top_10_confirmed = latest_data.nlargest(10, 'pct_confirmed')
  top_10_deaths = latest_data.nlargest(10, 'pct_deaths')
  top_10_deaths_wrt_cases = latest_data.nlargest(10, 'pct_deaths_wrt_confirmed')
  top_state = list(top_10_confirmed['state'])
  fig = go.Figure()
  fig.add_trace(go.Bar(x=top_10_confirmed['state'], y=top_10_confirmed['pct_confirmed'], name='% Confirmed', text=top_10_confirmed['pct_confirmed'], textposition='auto'))
  fig.add_trace(go.Bar(x=top_10_confirmed['state'], y=top_10_confirmed['pct_deaths_wrt_confirmed'], name='Mortality Rate', text=top_10_confirmed['pct_deaths_wrt_confirmed'], textposition='auto'))
  #fig.add_trace(go.Bar(x=top_10_deaths_wrt_cases['state'], y=top_10_deaths_wrt_cases['pct_deaths_wrt_confirmed'], name='% Deaths w.r.t Cases'))
  fig.show()
  return top_state


state = states_rates() # top 10 states w.r.t confirmed cases
def state_confirmed():
  fig = go.Figure()
  fig_log = go.Figure()
  fig_rate = go.Figure()
  fig_daily = go.Figure()
  for i in state:
    df_state = df_us_time_series[df_us_time_series['state']==i]
    print (df_state)
    df_state = df_state[df_state['cases']>1000]
    df_state.reset_index(inplace=True)
    df_state.drop(columns=['index'])
    df_state.index = df_state.index + 1
    label = [round(m/1000, 2) for m in df_state['cases']]
    x = list(df_state.index)

    # Calculate Rate of Confirmation
    r, c = df_state.shape
    xy = []
    daily_confirmed = []
    for j in range(r-1):
      temp = (df_state.iloc[j+1, 4] - df_state.iloc[j, 4] )/ df_state.iloc[j, 4]
      temp_daily = df_state.iloc[j+1, 4] - df_state.iloc[j, 4] # daily
      xy.append(temp)
      daily_confirmed.append(temp_daily)
    
    # Create Trace
    fig.add_trace(go.Scatter(x=x, y=df_state['cases'], mode='lines+markers', name=i, \
                             text=label, textposition='top left'))
    fig_rate.add_trace(go.Scatter(x=x, y=xy, mode='lines+markers', name=i, \
                             textposition='top left'))
    fig_log.add_trace(go.Scatter(x=x, y=df_state['cases'], mode='lines+markers', name=i, \
                             textposition='top left'))
    
    fig_daily.add_trace(go.Scatter(x=x, y=daily_confirmed, mode='lines', name=i, \
                             textposition='top left', line={"shape": "spline", "smoothing": 0.5}))
  
  fig_log.update_layout(title='Confirmed Cases by State',
                    xaxis_title='Number of Days since 100 Confirmed Cases',
                    yaxis_title= 'Confirmed Cases - Log',
                    yaxis_type='log'
                    )
  
  fig.update_layout(title='Confirmed Cases by State',
                  xaxis_title='Number of Days since 100 Confirmed Cases',
                  yaxis_title= 'Confirmed Cases'
                  )
  
  fig_rate.update_layout(title='Rate of Confirmed Cases by State',
                  xaxis_title='Number of Days since 100 Confirmed Cases',
                  yaxis_title= 'Rate of Confirmed Cases'
                  )
  
  fig_daily.update_layout(title='Daily Confirmed Cases by State',
                  xaxis_title='Number of Days since 100 Confirmed Cases',
                  yaxis_title= 'Daily Confirmed Cases'
                  )
  
 
  
  fig.show()
  fig_log.show()
  fig_rate.show()
  fig_daily.show()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [13]:
covid_core_metrics(country)
days_since_100_confirmed()
state_confirmed()

            date     state  fips   cases  deaths
246   2020-03-01  New York    36       1       0
261   2020-03-02  New York    36       1       0
276   2020-03-03  New York    36       2       0
293   2020-03-04  New York    36      11       0
313   2020-03-05  New York    36      22       0
338   2020-03-06  New York    36      44       0
369   2020-03-07  New York    36      89       0
404   2020-03-08  New York    36     106       0
440   2020-03-09  New York    36     142       0
478   2020-03-10  New York    36     173       0
521   2020-03-11  New York    36     217       0
568   2020-03-12  New York    36     326       0
618   2020-03-13  New York    36     421       0
669   2020-03-14  New York    36     610       2
722   2020-03-15  New York    36     732       6
775   2020-03-16  New York    36     950      10
828   2020-03-17  New York    36    1374      17
882   2020-03-18  New York    36    2382      27
936   2020-03-19  New York    36    4152      30
990   2020-03-20  Ne

In [15]:
df_us_time_series[df_us_time_series['state']=='New York']

,date,state,fips,cases,deaths
246,2020-03-01,New York,36,1,0
261,2020-03-02,New York,36,1,0
276,2020-03-03,New York,36,2,0
293,2020-03-04,New York,36,11,0
313,2020-03-05,New York,36,22,0
338,2020-03-06,New York,36,44,0
369,2020-03-07,New York,36,89,0
404,2020-03-08,New York,36,106,0
440,2020-03-09,New York,36,142,0
478,2020-03-10,New York,36,173,0


In [57]:
'''
# pdf table extract
!pip install tabula-py
import tabula as tb
# CT Data
df_ct = tb.read_pdf("https://portal.ct.gov/-/media/Coronavirus/CTDPHCOVID19summary3242020.pdf", pages='all', stream=True)[1]

# NYC Data
df_nyc = tb.read_pdf("https://www1.nyc.gov/assets/doh/downloads/pdf/imm/covid-19-daily-data-summary.pdf", pages='all', stream=True)[0]
'''

'\n# pdf table extract\n!pip install tabula-py\nimport tabula as tb\n# CT Data\ndf_ct = tb.read_pdf("https://portal.ct.gov/-/media/Coronavirus/CTDPHCOVID19summary3242020.pdf", pages=\'all\', stream=True)[1]\n\n# NYC Data\ndf_nyc = tb.read_pdf("https://www1.nyc.gov/assets/doh/downloads/pdf/imm/covid-19-daily-data-summary.pdf", pages=\'all\', stream=True)[0]\n'

In [0]:
# Hospitalization and Testing Charts

# Data from COVID Tracking

df_state_daily = pd.read_csv("http://covidtracking.com/api/states/daily.csv")
df_counties_daily = pd.read_csv("https://covidtracking.com/api/counties.csv")

# for Countries

df_country_testing_source = pd.read_excel('https://github.com/owid/covid-19-data/blob/master/public/data/testing/covid-testing.xlsx?raw=true')
df_country_testing = df_country_testing_source[['Entity', 'Date', 'Cumulative total', 'Daily change in cumulative total']].copy()

In [59]:
df_state_daily.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,recovered,hash,dateChecked,death,hospitalized,total,totalTestResults,posNeg,fips,deathIncrease,hospitalizedIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease
0,20200407,AK,213.0,6700.0,NaN,NaN,23.0,NaN,NaN,NaN,NaN,29.0,427f23b794025bcc59ba746dc6721ba1ae4f6ff9,2020-04-07T20:00:00Z,6.0,23.0,6913,6913,6913,2,0.0,0.0,8.0,22.0,30.0
1,20200407,AL,2119.0,12797.0,NaN,NaN,271.0,NaN,NaN,NaN,NaN,NaN,bc437164edce7423a9a6a0095632666a06d512c4,2020-04-07T20:00:00Z,56.0,271.0,14916,14916,14916,1,6.0,31.0,0.0,151.0,151.0
2,20200407,AR,946.0,12692.0,NaN,74.0,148.0,NaN,NaN,26.0,43.0,142.0,3091efb8651506655cf6538a5a63244288d812e8,2020-04-07T20:00:00Z,16.0,148.0,13638,13638,13638,5,0.0,11.0,722.0,71.0,793.0
3,20200407,AS,0.0,20.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7c2f821451b256479a66373d658c658912e64f9f,2020-04-07T20:00:00Z,0.0,NaN,31,20,20,60,0.0,0.0,0.0,0.0,0.0
4,20200407,AZ,2575.0,30800.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,943f82c1a910748ffe6ec553b93b7b964125cc38,2020-04-07T20:00:00Z,73.0,NaN,33375,33375,33375,4,8.0,0.0,722.0,119.0,841.0


In [0]:
df_daily_testing = df_state_daily.groupby('date')['totalTestResultsIncrease'].sum().to_frame()
df_hospitalization = df_state_daily.groupby('date')['hospitalized'].sum().to_frame()
df_hospitalization['date'] = df_hospitalization.index
df_daily_testing['date'] = df_daily_testing.index
df_daily_testing['date_time'] = pd.to_datetime(df_daily_testing['date'], format='%Y%m%d', errors='ignore')
df_hospitalization['date_time'] = pd.to_datetime(df_hospitalization['date'], format='%Y%m%d', errors='ignore')

In [0]:
#df_state_daily.groupby('date', as_index=False)['positiveIncrease'].sum()

In [0]:
def rowCalculation(input_df, i):
  # accepts input df and the column for which row calculations are needed
  # returns the calculations in a list
  r, c = input_df.shape
  xy = [0]
  for j in range(r-1):
    temp = (input_df.iloc[j+1, i] - input_df.iloc[j, i] )
    xy.append(temp)
  return xy

df_hospitalization['changeInHospitalization'] = rowCalculation(df_hospitalization, 0)

In [0]:
latest_day = df_state_daily['date'].max()
todays_data_df = df_state_daily[df_state_daily['date']==latest_day]
pop = pd.read_csv('/content/drive/My Drive/Colab Notebooks/statepop.csv') # upload population
todays_data_df = pd.merge(todays_data_df, pop, on='state', how='left') # get population
todays_data_df['per_capital_testing'] = todays_data_df['totalTestResults'] / todays_data_df['pop']
# top 10 and bottom 10
top_10_testing = todays_data_df.nlargest(10, 'per_capital_testing')
bottom_10_testing = todays_data_df.nsmallest(10, 'per_capital_testing')

In [64]:
top_10_testing

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,recovered,hash,dateChecked,death,hospitalized,total,totalTestResults,posNeg,fips,deathIncrease,hospitalizedIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease,pop,per_capital_testing
37,20200407,NY,138863.0,201195.0,NaN,17493.0,32083.0,4593.0,NaN,NaN,NaN,14590.0,15947b1c9ed54db71e224d5882c2e146b415b520,2020-04-07T20:00:00Z,5489.0,32083.0,340058,340058,340058,36,731.0,1880.0,11073.0,8174.0,19247.0,19453561,0.017481
20,20200407,LA,16284.0,58371.0,NaN,1996.0,NaN,NaN,NaN,519.0,NaN,NaN,9139f6e23f09c887df221642ce10c85516944f49,2020-04-07T20:00:00Z,582.0,NaN,74655,74655,74655,22,70.0,0.0,4072.0,1417.0,5489.0,4648794,0.016059
52,20200407,WA,8384.0,83391.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f9019f6fa9a2ec068873ccb09e6dadc7d755cc24,2020-04-07T20:00:00Z,372.0,NaN,91775,91775,91775,53,34.0,0.0,0.0,400.0,400.0,7614893,0.012052
21,20200407,MA,15202.0,66142.0,NaN,NaN,1435.0,NaN,NaN,NaN,NaN,NaN,480c678f573f195caec6a15c44dcfe844e95ef3c,2020-04-07T20:00:00Z,356.0,1435.0,81344,81344,81344,25,96.0,194.0,3550.0,1365.0,4915.0,6892503,0.011802
51,20200407,VT,575.0,6554.0,NaN,29.0,45.0,NaN,NaN,NaN,NaN,15.0,44493af709005398cd2f24b0c1c9f7cf96707f38,2020-04-07T20:00:00Z,23.0,45.0,7129,7129,7129,50,0.0,0.0,464.0,32.0,496.0,623989,0.011425
8,20200407,DC,1211.0,6612.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,318.0,4bb7cd953c61db17cc89b08b77d926f2eea787e6,2020-04-07T20:00:00Z,22.0,NaN,7823,7823,7823,11,-2.0,0.0,256.0,114.0,370.0,705749,0.011085
48,20200407,UT,1738.0,32909.0,NaN,NaN,148.0,NaN,NaN,NaN,NaN,NaN,40a68524a0d0c4244c5e3929960fac5865a4b540,2020-04-07T20:00:00Z,13.0,148.0,34647,34647,34647,49,0.0,10.0,1190.0,63.0,1253.0,3205958,0.010807
34,20200407,NJ,44416.0,50558.0,NaN,7017.0,NaN,1651.0,NaN,1540.0,NaN,NaN,cc2566fc957facb8d98c9cdcaafbb8235297c5e6,2020-04-07T20:00:00Z,1232.0,NaN,94974,94974,94974,34,229.0,0.0,2616.0,3326.0,5942.0,8882190,0.010693
35,20200407,NM,686.0,21139.0,NaN,48.0,NaN,NaN,NaN,18.0,NaN,133.0,74b47d956a13a1210784d310796b4b88918963db,2020-04-07T20:00:00Z,12.0,NaN,21825,21825,21825,35,0.0,0.0,2627.0,62.0,2689.0,2096829,0.010409
31,20200407,ND,237.0,7466.0,NaN,18.0,33.0,NaN,NaN,NaN,NaN,82.0,59b258409a58763928c3aa0087984baad4dd1600,2020-04-07T20:00:00Z,4.0,33.0,7703,7703,7703,38,1.0,1.0,478.0,12.0,490.0,762062,0.010108


In [86]:
fig = go.Figure()
fig2 = go.Figure()
fig3 = go.Figure()
fig4 = go.Figure()
#fig.add_trace(go.Scatter(x=df_daily_testing['date_time'], y=df_daily_testing['totalTestResultsIncrease']))
fig.add_trace(go.Bar(x=df_daily_testing['date_time'], y=df_daily_testing['totalTestResultsIncrease'], text=df_daily_testing['totalTestResultsIncrease'], textposition='auto'))
fig2.add_trace(go.Scatter(x=df_hospitalization['date_time'], y=df_hospitalization['changeInHospitalization']))
fig3.add_trace(go.Bar(y=top_10_testing['state'], x=top_10_testing['per_capital_testing']*1000, orientation='h', text=top_10_testing['totalTestResults']))
fig4.add_trace(go.Bar(y=bottom_10_testing['state'], x=bottom_10_testing['per_capital_testing']*1000, orientation='h'))

# Add SK benchmark line
fig3.add_shape(
        # Line Vertical
        dict(
            type="line",
            x0=9,
            y0=0,
            x1=9,
            y1=10,
            line=dict(
                color="Red",
                width=3
            )
))

fig.update_layout(title='Daily Testing',
                  xaxis_title='Days',
                  yaxis_title= 'Tests Conducted'
                  )

fig2.update_layout(title='Daily Hospitalization',
                  xaxis_title='Days',
                  yaxis_title= 'Hospitalization'
                  )

fig3.update_layout(title='Top 10 States By Per Capita Testing',
                  xaxis_title='Testing Per 1000',
                  yaxis_title= 'States'
                  )

fig4.update_layout(title='Bottom 10 States By Per Capita Testing',
                  xaxis_title='Testing per 1000',
                  yaxis_title= 'States'
                  )


fig.show()
fig2.show()
fig3.show()
fig4.show()

In [66]:
      todays_data_df.shape

(56, 27)

In [67]:
pop.shape

(56, 2)

In [68]:
(466804 / 51269185)

0.009104962366770605

In [82]:
# Total US
total_us_testing = todays_data_df.groupby('date')['totalTestResults'].sum()
print (total_us_testing)
per_capita = total_us_testing / 331002651
print (per_capita*1000)
print (1/per_capita)

date
20200407    2054462
Name: totalTestResults, dtype: int64
date
20200407    6.206784
Name: totalTestResults, dtype: float64
date
20200407    161.114029
Name: totalTestResults, dtype: float64


date
20200407    1027231.0
Name: totalTestResults, dtype: float64

In [70]:
1917095/300000000

0.006390316666666666